# Scraper para UDG Zapotlán

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import csv
import time
import random
from datetime import datetime

# Configuración
BASE_URL = "https://udgtv.com/"
SECTION_URL = "https://udgtv.com/noticias/zapotlan-el-grande-noticias/"
OUTPUT_FILE = f"udgtv_zapotlan_new_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
MAX_LOAD_MORE = 20
DELAY = random.uniform(1, 3)

# Configurar Chrome
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

def setup_driver():
    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(10)
        return driver
    except Exception as e:
        print(f"Error al configurar el driver: {str(e)}")
        return None

def clean_text(text):
    return ' '.join(text.strip().split()) if text else ""

def extract_article_data(driver, article_url):
    print(f"\nExtrayendo: {article_url}")
    try:
        driver.get(article_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//h1[@class="entry-title"]'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Selectores precisos
        title = clean_text(soup.select_one('h1.entry-title').get_text())
        
        date_elem = soup.select_one('span.posted-on time') or soup.select_one('span.posted-on')
        date = clean_text(date_elem.get_text()) if date_elem else ""
        
        author_elem = soup.select_one('div.td-post-author-name a')
        author = clean_text(author_elem.get_text()) if author_elem else ""
        
        content_div = soup.select_one('div.td-post-content')
        paragraphs = []
        
        if content_div:
            for element in content_div(['script', 'style', 'iframe']):
                element.decompose()
            
            for elem in content_div.find_all(['p', 'h2', 'h3']):
                text = clean_text(elem.get_text())
                if text and len(text.split()) > 2:
                    paragraphs.append(text)
        
        content = '\n\n'.join(paragraphs) if paragraphs else "Contenido no disponible"
        
        print(f"✓ Título: {title}")
        print(f"✓ Fecha: {date}")
        print(f"✓ Autor: {author}")
        print(f"✓ Caracteres: {len(content)}")
        
        return {
            'titulo': title,
            'articulo': content,
            'fecha': date,
            'autor': author,
            'url': article_url,
            'categoria': 'Zapotlán el Grande'
        }
        
    except Exception as e:
        print(f"Error procesando artículo: {str(e)}")
        return None

def get_all_article_links(driver):
    print("\nCargando página inicial...")
    driver.get(SECTION_URL)
    time.sleep(5)
    
    # Aceptar cookies si es necesario
    try:
        cookie_btn = driver.find_element(By.ID, "cookie_action_close_header")
        cookie_btn.click()
        print("Cookies aceptadas")
        time.sleep(1)
    except:
        pass
    
    articles = []
    
    # Selectores múltiples para encontrar artículos
    selectors = [
        'div.td-module-thumb a',  # Para los artículos en div.td-module-thumb
        'div.td-module-container h3 a',  # Para los artículos en los divs que mencionaste
        'div.post-item h3 a',  # Selector alternativo
        'article.post h2 a'  # Otro selector alternativo
    ]
    
    for selector in selectors:
        try:
            found_articles = driver.find_elements(By.CSS_SELECTOR, selector)
            new_links = [a.get_attribute('href') for a in found_articles if a.get_attribute('href') and a.get_attribute('href') not in articles]
            articles.extend(new_links)
            print(f"Con selector '{selector}' encontré {len(new_links)} artículos nuevos")
        except Exception as e:
            print(f"Error con selector {selector}: {str(e)}")
    
    print(f"Total inicial: {len(articles)} artículos")
    
    # Clicar en "Ver Más"
    for i in range(MAX_LOAD_MORE):
        try:
            ver_mas_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Ver Más")]'))
            )
            driver.execute_script("arguments[0].scrollIntoView();", ver_mas_btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", ver_mas_btn)
            print(f"Clic en 'Ver Más' ({i+1}/{MAX_LOAD_MORE})")
            time.sleep(3)
            
            # Buscar nuevos artículos después de cada clic
            current_count = len(articles)
            for selector in selectors:
                try:
                    new_articles = driver.find_elements(By.CSS_SELECTOR, selector)
                    new_links = [a.get_attribute('href') for a in new_articles if a.get_attribute('href') and a.get_attribute('href') not in articles]
                    articles.extend(new_links)
                except:
                    continue
            
            if len(articles) == current_count:
                print("No se encontraron nuevos artículos. Terminando...")
                break
                
            print(f"Total después de clic {i+1}: {len(articles)} artículos")
            
        except Exception as e:
            print(f"Error al cargar más artículos: {str(e)}")
            break
    
    return list(set(articles))

def main():
    driver = setup_driver()
    if not driver:
        return
    
    try:
        with open(OUTPUT_FILE, mode='w', encoding='utf-8', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['titulo', 'articulo', 'fecha', 'autor', 'url', 'categoria'])
            writer.writeheader()
            
            start_time = time.time()
            article_links = get_all_article_links(driver)
            
            print(f"\nTotal de artículos a procesar: {len(article_links)}")
            articles_scraped = 0
            
            for link in article_links:
                article_data = extract_article_data(driver, link)
                if article_data:
                    writer.writerow(article_data)
                    csvfile.flush()
                    articles_scraped += 1
                
                time.sleep(DELAY)
            
            duration = time.time() - start_time
            print(f"\nScraping completado. Artículos extraídos: {articles_scraped}")
            print(f"Archivo guardado: {OUTPUT_FILE}")
            print(f"Tiempo total: {duration//60:.0f}m {duration%60:.0f}s")
            
    finally:
        driver.quit()
        print("\nNavegador cerrado")

if __name__ == "__main__":
    print("""
    ============================================
    SCRAPER UDGTV - SELECTORES PRECISOS
    Basado en análisis de estructura HTML
    ============================================
    """)
    
    main()


    SCRAPER UDGTV - SELECTORES PRECISOS
    Basado en análisis de estructura HTML
    

Cargando página inicial...
Con selector 'div.td-module-thumb a' encontré 34 artículos nuevos
Con selector 'div.td-module-container h3 a' encontré 0 artículos nuevos
Con selector 'div.post-item h3 a' encontré 0 artículos nuevos
Con selector 'article.post h2 a' encontré 0 artículos nuevos
Total inicial: 34 artículos
Clic en 'Ver Más' (1/20)
Total después de clic 1: 44 artículos
Clic en 'Ver Más' (2/20)
Total después de clic 2: 54 artículos
Clic en 'Ver Más' (3/20)
Total después de clic 3: 64 artículos
Clic en 'Ver Más' (4/20)
Total después de clic 4: 74 artículos
Clic en 'Ver Más' (5/20)
Total después de clic 5: 84 artículos
Clic en 'Ver Más' (6/20)
Total después de clic 6: 94 artículos
Clic en 'Ver Más' (7/20)
Total después de clic 7: 104 artículos
Clic en 'Ver Más' (8/20)
Total después de clic 8: 114 artículos
Clic en 'Ver Más' (9/20)
Total después de clic 9: 124 artículos
Clic en 'Ver Más' (10/20)